Análise de sentimentos: classificação de textos em um sentimento predefinido.
Biblioteca utilizada: NLTK: Natural Language Toolkit.
Serão utilizado um conjunto de dados de tweets, que passarão por uma limpeza. Após isso, o modelo será treinado e usado para classificar os tweets em sentimentos negativos e positivos.

Comandos utilizados para baixar os tweets:
```
pip install nltk==3.3
python
imnport nltk
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
```

## Tokeninzando os dados

In [1]:
from nltk.corpus import twitter_samples

**strings()**: imprime os tweets em um conjunto de dados como sequência de caracteres. Facilita o processamento e o teste.

In [2]:
# separando entre categorias de tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

**punkt**: modelo pré-treinado que ajuda a tokenizar palavras e frases.

In [3]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
# print(tweet_tokens[0])

## Normalizando os dados

**Normalização**: converter a palavra para sua forma canônica. Ajuda a agrupar palavras com o mesmo significado, mas com formas diferentes.


Técnicas utilizadas: *stemming* e *lematização*.

**stemming**: remoção dos afixos de uma palavra. Remove os fins das palavras.

**lematização:**: normaliza uma palavra de acordo com o contexto e a análise morfológica da palavra. Tem um custo de velocidade.

**wordnet**: banco de dados lexical para inglês que ajuda o script a determinar a raiz da palavra.

**averaged_perceptron_tagger**: para determinar o contexto de uma palavra numa frase.

É necessário determinar o contexto das palavras através de um **algoritmo de marcação**: avalia a posição relativa de uma palavra em uma frase. Será utilizado o *pos_tag*.

In [4]:
from nltk.tag import pos_tag
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


Significado das tags:
* **NNP**: substantivo, próprio, singular
* **NN**: substantivo, comum, singular ou de massa
* **IN**: preposição ou conjunção, subordinação
* **VBG**: partícula verbal, gerúndio ou presente
* **VBN**: verbo, particípio passado

In [5]:
# eliminando uma frase
from nltk.stem.wordnet import WordNetLemmatizer
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = [] 
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


Nota-se que os verbos foram para a forma básica.

**lematize_sentence**: obtém a tag de posição de cada token de um tweet.

## Removendo os ruídos dos dados

Removendo qualquer coisa que não adicione informação ao texto, depende do texto.

**sub()**: para substituir um termo por uma sequência vazia.

In [6]:
import re, string
def remove_noise(tweet_tokens, stop_words =()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\'(?:%[0-9a-fA-F][0-9a-fA-F]))+",'', token)
        token = re.sub("(@[A-Za-z0-9_]+)","",token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

**remove_noise()**: remove o ruído e incorpora a normalização e a lematização. Remove os hiperlinks, os @ ou _ além das pontuações.

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(remove_noise(tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [8]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [9]:
 # comparando os tweets positivos
 print(positive_tweet_tokens[500])
 print(positive_cleaned_tokens_list[500])


['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


## Determinando a densidade da palavra

**get_all_words**: recebe um lista de tweets como argumento para fornecer uma lista de palavras em todos os tokens de tweet associados.

In [10]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token
all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [11]:
# printando as 10 palavras mais comuns em tweets positivos

from nltk import FreqDist
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [13]:
all_neg_words = get_all_words(negative_cleaned_tokens_list)
freq_dist_neg = FreqDist(all_neg_words)
print(freq_dist_neg.most_common(10))

[(':(', 4585), (':-(', 501), ("i'm", 343), ('...', 332), ('get', 325), ('miss', 291), ('go', 275), ('please', 275), ('want', 246), ('like', 218)]


**Resumo do que foi feito:**
1. Tweets foram extraídos;
2. Tweets foram tokenizados;
3. Tweets foram normalizados;
4. Tweets foram limpos;
5. Análise de frequência de tokens nos tweets.

## Preparando dados para o modelo

Os tokens serão convertidos para dicionário e depois divididos para fins de treinamento e teste.

### Convertendo os tokens em um dicionário

In [16]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

### Divisão do conjunto de dados para treinamento e teste do modelo

Anexando um rótulo Positive ou Negative a cada tweet e depois criando um dataset com ambos os conjuntos. Depois os tweets são embaralhados dentre desse conjunto maior para que não haja enviesamento no treinamento.

A seguir, é feita uma separação entre dados de treino e dados de teste.

In [17]:
import random
positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset
random.shuffle(dataset)
train_data = dataset[:7000]
test_data = dataset[7000:]

### Criando e testando o modelo

In [18]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is: ", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is:  0.9956666666666667
Most Informative Features
                      :( = True           Negati : Positi =   2065.0 : 1.0
                      :) = True           Positi : Negati =   1647.1 : 1.0
                     sad = True           Negati : Positi =     23.3 : 1.0
                follower = True           Positi : Negati =     22.3 : 1.0
                     bam = True           Positi : Negati =     19.4 : 1.0
                  arrive = True           Positi : Negati =     18.4 : 1.0
                 welcome = True           Positi : Negati =     15.7 : 1.0
               community = True           Positi : Negati =     14.2 : 1.0
                    glad = True           Positi : Negati =     14.0 : 1.0
                     ugh = True           Negati : Positi =     13.2 : 1.0
None


Agora testando o modelo com tweets aleatórios.

In [27]:
from nltk.tokenize import word_tokenize
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token,True] for token in custom_tokens)))

Negative


In [24]:
from nltk.tokenize import word_tokenize
custom_tweet = "Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies"
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token,True] for token in custom_tokens)))

Positive


A máquina não entende sarcasmo:

In [25]:
from nltk.tokenize import word_tokenize
custom_tweet = "Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline"
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token,True] for token in custom_tokens)))

Positive
